In [1]:
!pip install -e projekt_paczki

Obtaining file:///C:/Users/1aaa2/PycharmProjects/NYPD_projekt/projekt_paczki
  Attempting uninstall: project-package
    Found existing installation: project-package 0.0.1
    Uninstalling project-package-0.0.1:
      Successfully uninstalled project-package-0.0.1
  Running setup.py develop for project-package


In [1]:
import pandas as pd
import os

from analizy import top_5_na_rok
from analizy import diff

### Krok 1: wczytywanie i obróbka danych: 
Najpierw wczytuję dane oraz je czyszczę. Do usunięcia są puste kolumny Unnamed: 66. Trzeba także ujednolicić format w jakim są podane nazwy krajów - w pliku Co2 nazwy te są zapisane wielkimi literami, a w pozostałych plikach w normalny sposób.

Niektóre nazwy muszą być poprawione ręcznie, dobrym przykładem jest 'BOSNIA AND HERCEGOVINA' wraz z 'BOSNIA & HERCEGOVINA' - do tego tworzę oddzielny słownik


In [2]:
#wczytywanie danych

#ścieżki do danych: 
GDP_path = r'Dane\GDP1.csv'
Pop1_path = r'Dane\API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv'
co2_path = r'Dane\fossil-fuel-co2-emissions-by-nation_csv.csv'

GDP1 = pd.read_csv(GDP_path, skiprows = 4)
Pop1 = pd.read_csv(Pop1_path, skiprows = 3)
co2 = pd.read_csv(co2_path)

#podany zakres lat:
poczatek = 1961
koniec = 2014

#czyszczenie danych
#usuwam niepotrzebne kolumny
GDP1.drop('Unnamed: 66',axis = 1, inplace = True)
Pop1.drop('Unnamed: 66',axis = 1, inplace = True)

#zmieniam nazwy krajów na same wielkie litery - tak by format we wszystkich tabelkach był identyczny
GDP1['Country Name'] = GDP1['Country Name'].str.upper()
Pop1['Country Name'] = Pop1['Country Name'].str.upper()


#czyszczenie danych ciąg dalszych - sprawdzam czy nazwy krajów się zgadzają we wszystkich plikach
print(len(set(GDP1['Country Name']).symmetric_difference(set(Pop1['Country Name']))))
print(len(set(GDP1['Country Name']).symmetric_difference(set(co2['Country']))))
#Zauważmy, że nazwy w GDP1 i Pop1 są w takim samym formacie, ale już co2 ma inny format. Trzeba nanieść ręczne poprawki

correction = {
    'EGYPT': 'EGYPT, ARAB REP.',
    'ITALY (INCLUDING SAN MARINO)': 'ITALY',
    'VIET NAM': 'VIETNAM',
    'ANTIGUA & BARBUDA': 'ANTIGUA AND BARBUDA',
    'BOSNIA & HERZEGOVINA': 'BOSNIA AND HERZEGOVINA',
    'FRANCE (INCLUDING MONACO)':'FRANCE',
    'ST. KITTS-NEVIS': 'ST. KITTS AND NEVIS',
    'VENEZUELA': 'VENEZUELA, RB',
    'TURKEY': 'TURKIYE',
    'YEMEN': 'YEMEN, REP.',
    'CHINA (MAINLAND)':'CHINA',
    'SLOVAKIA': 'SLOVAK REPUBLIC',
    'UNITED STATES OF AMERICA': 'UNITED STATES OF AMERICA',
    'MACEDONIA': 'NORTH MACEDONIA',
    'PLURINATIONAL STATE OF BOLIVIA': 'BOLIVIA',
    'CONGO': 'CONGO, DEM. REP.'
    
}
co2['Country'].replace(correction, inplace = True)

0
200


### Krok 2: wybieram te lata, które są we wszystkich trzech tabelach

In [3]:
#wybór lat:
GDP_yrs = [int(x) for x in list(GDP1.columns[4:])]
Pop_yrs = [int(x) for x in list(Pop1.columns[4:])]
co2_yrs = [int(x) for x in list(co2['Year'])]

#bierzemy tylko te lata które są we wszystkich tabelach oraz mieszczą się w podanym przedziale 
yrs = list(set(GDP_yrs).intersection(set(Pop_yrs)).intersection(set(co2_yrs)))
yrs = sorted(filter(lambda x: x>=poczatek and x<=koniec, yrs))

if len(yrs) == 0: print('W podanym przedziale lat brak danych')

co2 = co2[co2.Year.isin(yrs)].reset_index(drop = True)
Pop1 = Pop1[list(Pop1.columns[:4])+[str(x)for x in yrs]]
GDP1 = GDP1[list(GDP1.columns[:4])+[str(x)for x in yrs]]

### Krok 3: scalam dane

Tworzę ramkę danych o nazwie 'Dane' zawierającą informacje ze wszystkich 3 ramek

In [4]:
#merge tabel
Pop1_melted = Pop1.melt(id_vars = ['Country Name','Country Code','Indicator Name','Indicator Code'], var_name = 'Year', value_name = 'Population')
GDP1_melted = GDP1.melt(id_vars = ['Country Name','Country Code','Indicator Name','Indicator Code'], var_name = 'Year', value_name = 'GDP')

Dane = Pop1_melted.merge(GDP1_melted, on = ['Country Name','Country Code','Year'], suffixes = ['_Pop','_GDP'])
Dane.rename(columns = {'Country Name':'Country'}, inplace = True)
Dane = Dane.astype({'Year':'int64'})
Dane = Dane.merge(co2, on = ['Country','Year'])
display(Dane)

,Country,Country Code,Indicator Name_Pop,Indicator Code_Pop,Year,Population,Indicator Name_GDP,Indicator Code_GDP,GDP,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total)
0,AFGHANISTAN,AFG,"Population, total",SP.POP.TOTL,1961,9169406.0,GDP (current US$),NY.GDP.MKTP.CD,5.488889e+08,134,48,80,0,6,0,0.01,0
1,ANGOLA,AGO,"Population, total",SP.POP.TOTL,1961,5531451.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,124,17,85,0,21,0,0.02,64
2,ALBANIA,ALB,"Population, total",SP.POP.TOTL,1961,1659800.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,622,88,496,23,16,0,0.37,0
3,UNITED ARAB EMIRATES,ARE,"Population, total",SP.POP.TOTL,1961,100801.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,3,0,3,0,0,0,0.03,0
4,ARGENTINA,ARG,"Population, total",SP.POP.TOTL,1961,20817270.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,13957,966,11496,1100,395,0,0.67,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8260,SAMOA,WSM,"Population, total",SP.POP.TOTL,2014,192220.0,GDP (current US$),NY.GDP.MKTP.CD,7.568060e+08,54,0,54,0,0,0,0.28,9
8261,"YEMEN, REP.",YEM,"Population, total",SP.POP.TOTL,2014,25823488.0,GDP (current US$),NY.GDP.MKTP.CD,4.322859e+10,6190,137,5090,581,381,0,0.24,153
8262,SOUTH AFRICA,ZAF,"Population, total",SP.POP.TOTL,2014,54544184.0,GDP (current US$),NY.GDP.MKTP.CD,3.811989e+11,133562,113068,16448,2404,1642,0,2.47,2945
8263,ZAMBIA,ZMB,"Population, total",SP.POP.TOTL,2014,15399793.0,GDP (current US$),NY.GDP.MKTP.CD,2.714102e+10,1228,132,797,0,299,0,0.08,33


### Krok 4: przeprowadzam analizy

a) Które kraje w danym roku generują najwięcej co2 na osobę? Tworzę tabelkę z pięcioma krajami o największej emisji w każdym roku. Jako wielkość emisji na osobę biorę kolumnę 'Per capita' która pochodzi z ramki co2 (alternatywny sposób - stworzyć nową kolumnę total/population, jednak wtedy wyniki odrobinę się różnią)

In [5]:
#Analizy - co2 na mieszkańca V2 
#co2 per capita
Dane['Co2_PC'] = Dane['Total']/Dane['Population']

co2_top_yearly = top_5_na_rok(Dane, 'Country','Co2_PC','Total')
display(co2_top_yearly)

,Year,Country,Co2_PC,Total
0,1961,LUXEMBOURG,0.009976,3161
1,1961,KUWAIT,0.009056,2722
2,1961,UNITED KINGDOM,0.003042,160605
3,1961,NEW CALEDONIA,0.002980,242
4,1961,CANADA,0.002898,52954
...,...,...,...,...
265,2014,QATAR,0.011960,29412
266,2014,CURACAO,0.010288,1604
267,2014,TRINIDAD AND TOBAGO,0.009263,12619
268,2014,KUWAIT,0.007049,26018


b) Które kraje w przeliczeniu na mieszkańca mają największy przychód? Tworzę tabelkę w analogiczny sposób jak poprzednio, jednak teraz celem będzie GDP per capita (które trzeba dodatkowo policzyć),

In [6]:
#Analizy - kraje z największym przychodem na mieszkańca V2
#GDP per capita
Dane['GDP_PC'] = Dane['GDP']/Dane['Population']

GDP_top_yearly = top_5_na_rok(Dane, 'Country','GDP_PC','GDP')
display(GDP_top_yearly)

,Year,Country,GDP_PC,GDP
0,1961,NEW ZEALAND,2343.292213,5.670064e+09
1,1961,SWEDEN,2288.921701,1.721269e+10
2,1961,CANADA,2240.433039,4.093495e+10
3,1961,LUXEMBOURG,2222.366366,7.041457e+08
4,1961,SWITZERLAND,1971.316323,1.071271e+10
...,...,...,...,...
265,2014,LIECHTENSTEIN,178864.851914,6.657171e+09
266,2014,LUXEMBOURG,123678.702143,6.880481e+10
267,2014,BERMUDA,98467.683994,6.413988e+09
268,2014,NORWAY,97019.182753,4.984101e+11


c) Kraje które najbardziej zwiększyły i zmniejszyły emisję co2 w ostatnich 10 latach (czyli w latach 2004-2014)

In [7]:
#Analizy - różnica w ostatnich 10 latach

co2_diff = diff(Dane, 2004, 2014, 'Country', 'Per Capita')

diff_lowest = co2_diff[:5]
diff_highest = co2_diff[-5:]

display(diff_lowest)
display(diff_highest)

Usunięto z powodu braku danych w podanych latach:
['LIECHTENSTEIN', 'SERBIA', 'MONTENEGRO', 'CURACAO', 'SUDAN']


,Country,Diff_Per Capita
8,ARUBA,-4.91
126,QATAR,-2.84
160,UNITED ARAB EMIRATES,-2.10
91,LUXEMBOURG,-2.08
55,FINLAND,-1.15


,Country,Diff_Per Capita
135,SINGAPORE,0.92
35,CHINA,0.95
102,MONGOLIA,1.02
109,NEW CALEDONIA,1.47
152,TRINIDAD AND TOBAGO,2.36
